In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
!pip install ktrain
!pip install tf_keras
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import ktrain
from ktrain import text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 74.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 60.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 3.5 MB/s eta 0:00:00
  Created wheel for ktrain: filename=ktrain-0.41.4-py3-none-any.whl size=25316932 sha256=7b032bcd94b95ebcd264406c569bb3b58b0d7244fdceb7d800c2c5149f11053e
  Stored in directory: /root/.cache/pip/wheels/fa/6a/9c/8a873b38bbd8bc207d33c64726bd18f7ef85f8e70dc3ac2e4b
  Created wheel for keras_bert: filename=ker

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 2 (False, Half True, Mostly False and Partly False)/fake_news_classification_mal_train.csv')
test_df = pd.read_excel('/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 2 (False, Half True, Mostly False and Partly False)/fake_news_classification_mal_test.xlsx')

In [ ]:
train_text = train_df['News']
train_label = train_df['Label']
test_text = test_df['News']

In [ ]:
train_df['Label'].value_counts()

,count
Label,
FALSE,1220
MOSTLY FALSE,295
FALSE,166
HALF TRUE,162
PARTLY FALSE,57


In [ ]:
classes_list = ["FALSE", "MOSTLY FALSE", "FALSE     ", "HALF TRUE", "PARTLY FALSE"]
label = train_label.apply(classes_list.index)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_text, label, test_size=0.2, random_state=42)

In [ ]:
len(X_train), len(y_train)

(1520, 1520)

In [ ]:
MODEL_NAME = 'FacebookAI/xlm-roberta-base'
t = text.Transformer(MODEL_NAME, maxlen=30, class_names=classes_list)
trn = t.preprocess_train(np.array(X_train), np.array(y_train))
val = t.preprocess_test(np.array(X_val), np.array(y_val))
model = t.get_classifier()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

preprocessing train...
language: ml
train sequence lengths:
	mean : 10
	95percentile : 17
	99percentile : 24


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: ml
test sequence lengths:
	mean : 9
	95percentile : 17
	99percentile : 22


In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = "Fake news task 2"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
learner.fit_onecycle(2e-5, 10, verbose=2, callbacks=callbacks_list)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.68421, saving model to Fake news task 2
48/48 - 100s - loss: 1.4717 - accuracy: 0.3724 - val_loss: 1.1168 - val_accuracy: 0.6842 - 100s/epoch - 2s/step
Epoch 2/10

Epoch 2: val_accuracy did not improve from 0.68421
48/48 - 15s - loss: 1.1661 - accuracy: 0.6316 - val_loss: 1.0186 - val_accuracy: 0.6842 - 15s/epoch - 314ms/step
Epoch 3/10

Epoch 3: val_accuracy did not improve from 0.68421
48/48 - 15s - loss: 1.1148 - accuracy: 0.6316 - val_loss: 0.9871 - val_accuracy: 0.6842 - 15s/epoch - 311ms/step
Epoch 4/10

Epoch 4: val_accuracy did not improve from 0.68421
48/48 - 15s - loss: 1.1052 - accuracy: 0.6316 - val_loss: 1.0082 - val_accuracy: 0.6842 - 15s/epoch - 307ms/step
Epoch 5/10

Epoch 5: val_accuracy did not improve from 0.68421
48/48 - 15s - loss: 1.0629 - accuracy: 0.6316 - val_loss: 0.9340 - val_accuracy: 0.6842 - 15s/epoch - 308ms/step
Epoch 6/10

Epoch 6: va

In [ ]:
learner.validate(class_names=t.get_classes())

12/12 [==============================] - 5s 137ms/step
              precision    recall  f1-score   support

       FALSE       0.82      0.76      0.79       260
MOSTLY FALSE       0.30      0.38      0.33        58
  FALSE            0.28      0.62      0.39        29
   HALF TRUE       0.00      0.00      0.00        23
PARTLY FALSE       0.00      0.00      0.00        10

    accuracy                           0.63       380
   macro avg       0.28      0.35      0.30       380
weighted avg       0.63      0.63      0.62       380



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


array([[198,  37,  25,   0,   0],
       [ 26,  22,  10,   0,   0],
       [  6,   5,  18,   0,   0],
       [  8,   8,   7,   0,   0],
       [  4,   2,   4,   0,   0]])

In [ ]:
test_text1 = test_df['News']
model.load_weights(filepath)
predictor = ktrain.get_predictor(learner.model, preproc=t)
predict = predictor.predict(test_text1.values)

In [ ]:
df = pd.DataFrame({
    'Id': test_df['S.no'],
    'Labels': predict
})

In [ ]:
import csv
df.to_csv("/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 2 (False, Half True, Mostly False and Partly False)/MNLP_task2_run1.csv", index=False)
check_submission = pd.read_csv("/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 2 (False, Half True, Mostly False and Partly False)/MNLP_task2_run1.csv")
check_submission.head()

,Id,Labels
0,FAKE_MAL_TE_0001,False
1,FAKE_MAL_TE_0002,False
2,FAKE_MAL_TE_0003,False
3,FAKE_MAL_TE_0004,False
4,FAKE_MAL_TE_0005,False
